In [1]:
# | default_exp examples

# ChatApp Examples

This notebook demonstrates various ways to use the ChatApp with full DI architecture.

In [2]:
from fasthtml.common import *
import asyncio
from pylogue.chatapp import ChatApp, ChatAppConfig, create_default_chat_app
from pylogue.session import InMemorySessionManager, Message, ChatSession
from pylogue.service import ChatService, ContextAwareResponder
from pylogue.renderer import ChatRenderer
from pylogue.cards import ChatCard

## Example 1: Simple Echo Bot

The simplest possible chat app - just echoes back what you say.

In [3]:
async def echo_bot(message: str, context=None) -> str:
    """Simple echo responder."""
    await asyncio.sleep(0.3)  # Simulate processing
    return f"🔊 You said: {message}"


# Create with one line
echo_app = create_default_chat_app(responder=echo_bot)

# To run: echo_app.run(port=5001)
print("✅ Echo bot ready!")

✅ Echo bot ready!


## Example 2: Custom Styled Chat

Customize colors, emojis, and styling.

In [4]:
# Define custom styling
custom_card = ChatCard(
    user_color="#2C1810",  # Dark brown
    assistant_color="#1A2332",  # Dark blue
    user_emoji="👨‍💼",
    assistant_emoji="🔮",
    width="70%",
    font_size="1.3em",
    border_radius="1.5em",
)

custom_config = ChatAppConfig(
    app_title="✨ Magic Assistant",
    page_title="Magic Chat",
    bg_color="#0f0f23",
)


async def magic_responder(message: str, context=None) -> str:
    """Mystical responder."""
    await asyncio.sleep(0.4)
    responses = [
        f"🔮 The spirits say: {message.upper()}",
        f"✨ Magic reveals: {message[::-1]}",  # Reversed
        f"🌟 The oracle speaks: {len(message)} mystical symbols detected!",
    ]
    import random

    return random.choice(responses)


magic_app = create_default_chat_app(
    responder=magic_responder, config=custom_config, card=custom_card
)

print("✅ Magic assistant ready!")

✅ Magic assistant ready!


## Example 3: Context-Aware Assistant

Uses conversation history to provide context-aware responses.

In [5]:
class SmartResponder:
    """Responder that tracks conversation context."""

    def __init__(self):
        self.keywords = {
            "hello": "👋 Hello! Nice to meet you!",
            "bye": "👋 Goodbye! Have a great day!",
            "help": "💡 I can help you with various tasks. Just ask!",
            "thanks": "😊 You're welcome!",
        }

    async def __call__(self, message: str, context=None) -> str:
        await asyncio.sleep(0.2)

        # Check conversation history
        if context and isinstance(context, list):
            message_count = len(context)

            # Check for keywords
            msg_lower = message.lower()
            for keyword, response in self.keywords.items():
                if keyword in msg_lower:
                    return f"{response}\n\n_We've exchanged {message_count} messages so far._"

        return f"📝 Interesting! You said: '{message}'"


# Context provider to give responder access to history
def provide_history(session: ChatSession):
    """Provide message history as context."""
    return session.get_messages()


smart_app = create_default_chat_app(
    responder=SmartResponder(),
    context_provider=provide_history,
    config=ChatAppConfig(app_title="🧠 Smart Assistant"),
)

print("✅ Smart assistant ready!")

✅ Smart assistant ready!


## Example 4: Multi-Language Code Assistant

Simulates a code assistant with syntax highlighting.

In [6]:
async def code_assistant(message: str, context=None) -> str:
    """Code-focused assistant."""
    await asyncio.sleep(0.5)

    msg_lower = message.lower()

    if "python" in msg_lower:
        return """Here's a Python example:

```python
def hello(name: str) -> str:
    return f"Hello, {name}!"

print(hello("World"))
```
"""
    elif "javascript" in msg_lower or "js" in msg_lower:
        return """Here's a JavaScript example:

```javascript
function hello(name) {
    return `Hello, ${name}!`;
}

console.log(hello("World"));
```
"""
    elif "help" in msg_lower:
        return """I can help with:
- Python code examples
- JavaScript code examples
- General programming questions

Just ask me about a language!
"""
    else:
        return f"💻 You asked: {message}\n\nTry asking about Python or JavaScript!"


code_app = create_default_chat_app(
    responder=code_assistant,
    config=ChatAppConfig(
        app_title="💻 Code Assistant",
        syntax_highlighting=True,
        highlight_langs=["python", "javascript", "typescript", "html", "css"],
    ),
    card=ChatCard(
        user_color="#1e1e1e",
        assistant_color="#252526",
        user_emoji="👨‍💻",
        assistant_emoji="🤖",
    ),
)

print("✅ Code assistant ready!")

✅ Code assistant ready!


## Example 5: Full Custom DI - Supply Chain Analyst

Complete custom implementation with specialized components.

In [7]:
class SupplyChainResponder:
    """Specialized responder for supply chain queries."""

    def __init__(self):
        self.knowledge = {
            "inventory": "📦 Inventory management tracks stock levels and movements.",
            "logistics": "🚚 Logistics coordinates transportation and delivery.",
            "demand": "📊 Demand forecasting predicts future needs.",
            "supplier": "🏭 Supplier management ensures reliable sourcing.",
            "rca": "🔍 Root Cause Analysis identifies problem origins.",
        }

    async def __call__(self, message: str, context=None) -> str:
        await asyncio.sleep(0.3)

        msg_lower = message.lower()

        # Check for known topics
        for topic, info in self.knowledge.items():
            if topic in msg_lower:
                return f"{info}\n\nWould you like to know more about {topic}?"

        # Default response with context
        if context and len(context) > 0:
            return f"📋 Analyzing: {message}\n\nBased on our conversation history, I'm tracking your supply chain queries."

        return f"🔍 Let me analyze: {message}\n\nAsk me about: inventory, logistics, demand, suppliers, or RCA."


# Custom initial messages
def supply_chain_initial():
    return [
        Message(
            role="Assistant",
            content="👋 Welcome to Supply Chain RCA Assistant!\n\nI can help you with:\n- 📦 Inventory issues\n- 🚚 Logistics problems\n- 📊 Demand analysis\n- 🏭 Supplier concerns\n\nWhat would you like to analyze?",
        )
    ]


# Create with full DI
supply_chain_app = ChatApp(
    session_manager=InMemorySessionManager(),
    chat_service=ChatService(
        responder=SupplyChainResponder(), context_provider=lambda s: s.get_messages()
    ),
    renderer=ChatRenderer(
        card=ChatCard(
            user_color="#1a3a1a",
            assistant_color="#1a1a3a",
            user_emoji="👤",
            assistant_emoji="🔍",
            width="75%",
        ),
        input_placeholder="Describe your supply chain issue...",
    ),
    config=ChatAppConfig(
        app_title="Supply Chain RCA Assistant",
        page_title="Supply Chain Analysis",
        bg_color="#0d1117",
        initial_messages_factory=supply_chain_initial,
    ),
)

print("✅ Supply chain assistant ready!")

✅ Supply chain assistant ready!


## Running Any Example

To run any of the apps above:

```python
# Choose your app
app = echo_app  # or magic_app, smart_app, code_app, supply_chain_app

# Run it
app.run(host="0.0.0.0", port=5001)
```

Then open your browser to `http://localhost:5001`

## Advanced: Adding Custom Routes

You can extend any app with custom routes:

In [8]:
# Get the underlying FastHTML app
fasthtml_app = supply_chain_app.get_app()


# Add a custom route
@fasthtml_app.route("/health")
def health_check():
    return {"status": "healthy", "app": "supply_chain"}


# Add another route
@fasthtml_app.route("/stats")
def stats():
    manager = supply_chain_app.session_manager
    return {
        "active_sessions": len(manager.list_sessions()),
        "app_type": "Supply Chain RCA",
    }


print("✅ Added custom routes: /health and /stats")

✅ Added custom routes: /health and /stats


## Testing Components Individually

In [9]:
# Test a responder
responder = SupplyChainResponder()
response = await responder("Tell me about inventory", None)
print(f"Responder test: {response[:50]}...")

Responder test: 📦 Inventory management tracks stock levels and mov...


In [10]:
# Test session management
from pylogue.session import InMemorySessionManager

manager = InMemorySessionManager()
session = manager.create_session("test-session")
session.add_message("User", "Test message")
session.add_message("Assistant", "Test response")

print(f"Session has {len(session)} messages")
print(f"Messages: {session.get_message_dicts()}")

Session has 2 messages
Messages: [{'role': 'User', 'content': 'Test message', 'id': '4d4d6a07-08a0-417d-9b10-541cd4fc9950'}, {'role': 'Assistant', 'content': 'Test response', 'id': '2cb9f2db-6b12-4918-ba5a-49c9e63dcd27'}]


In [11]:
# Test renderer
from fasthtml.jupyter import render_ft

render_ft()

test_messages = [
    Message(role="User", content="Hello!"),
    Message(role="Assistant", content="Hi! How can I help?"),
]

renderer = ChatRenderer(card=ChatCard(user_color="#2a2a2a", assistant_color="#3a3a3a"))
renderer.render_messages(test_messages)

<div>
  <div id="chat-cards" class="chat-cards" style="display: flex; flex-direction: column; gap: 10px;">
    <div style="background: #2a2a2a; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hello!</div>
    </div>
    <div style="background: #3a3a3a; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🕵️‍♂️ Assistant: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hi! How can I help?</div>
    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>
